In [ ]:
from nbdev import *
%nbdev_default_export bayesian
%load_ext autoreload
%autoreload 2
from utilities.ipynb_docgen import *
!date

Cells will be exported to wtlike.bayesian,
unless a different module is specified after an export flag: `%nbdev_export special.module`
Sun Apr 25 06:21:48 PDT 2021


# Bayesian Blocks

> Partition a light curve with the Bayesian Block algorithm

The functions implemented here are:

- `get_bb_partition` to perform a BB partition. It requires a "fitness" function to perform an evaluation of the 
likelihoods for a set of sequential cells. There are two such, using the number of counts, and the Kerr likelihood. See the [Bayesian Block reference](https://arxiv.org/pdf/1207.5578.pdf).

 - `CountFitness`
 - `LikelihoodFitness`, the default
 

- `bb_overplot` which overplots fits to the partitioned blocks on the original light curve.


In [ ]:
%nbdev_export
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.stats.bayesian_blocks import FitnessFunc

from wtlike.config import *
from wtlike.lightcurve import * #get_lightcurve, fit_cells, flux_plot
from wtlike.cell_data import * #get_cells, partition_cells
from wtlike.loglike import *


In [ ]:
#export
class CountFitness(FitnessFunc):
    """
    Adapted version of a astropy.stats.bayesian_blocks.FitnessFunc
    Considerably modified to give the `fitness function` access to the cell data.
    
    Implements the Event model using exposure instead of time.

    """

    def __init__(self, lc, p0=0.05,):
        """
        - lc  -- a LightCurve data table, with  exposure (e) and counts (n),
            as well as a representation of the likelihood for each cell
        - p0 --
        """
        self.p0=p0
        self.df= df= lc
        N = self.N = len(df)
        # Invoke empirical function from Scargle 2012
        self.ncp_prior = self.p0_prior(N)

        #actual times for bin edges
        t = df.t.values
        dt = df.tw.values/2
        self.mjd = np.concatenate([t-dt, [t[-1]+dt[-1]] ] ) # put one at the end
        self.name = self.__class__.__name__
        self.setup()

    def setup(self):
        df = self.df

        # counts per cell
        self.nn = df.n.values
        assert min(self.nn)>0, 'Attempt to Include a cell with no contents'

        # edges and block_length use exposure as "time"
        e = df.e.values
        self.edges = np.concatenate([[0], np.cumsum(e)])
        self.block_length = self.edges[-1] - self.edges

    def __str__(self):
        
        return f'{self.name}: {self.N} cells, spanning {self.block_length[0]:.1f} days, prior={self.ncp_prior:.1f}'
        
    def __call__(self, R):
        """ The fitness function needed for BB algorithm
        For cells 0..R return array of length R+1 of the maximum log likelihoods for combined cells
        0..R, 1..R, ... R
        """
        # exposures and corresponding counts
        w_k = self.block_length[:R + 1] - self.block_length[R + 1]
        N_k = np.cumsum(self.nn[:R + 1][::-1])[::-1]

        # Solving eq. 26 from Scargle 2012 for maximum $\lambda$ gives
        return N_k * (np.log(N_k) - np.log(w_k))

    def fit(self):
        """Fit the Bayesian Blocks model given the specified fitness function.
        Refactored version using code from bayesian_blocks.FitnesFunc.fit
        Returns
        -------
        edges : ndarray
            array containing the (M+1) edges, in MJD units, defining the M optimal bins
        """
        # This is the basic Scargle algoritm, copied almost verbatum
        # ---------------------------------------------------------------

        # arrays to store the best configuration
        N = self.N
        best = np.zeros(N, dtype=float)
        last = np.zeros(N, dtype=int)

        # ----------------------------------------------------------------
        # Start with first data cell; add one cell at each iteration
        # ----------------------------------------------------------------
        for R in range(N):

            # evaluate fitness function
            fit_vec = self(R)

            A_R = fit_vec - self.ncp_prior
            A_R[1:] += best[:R]

            i_max = np.argmax(A_R)
            last[R] = i_max
            best[R] = A_R[i_max]

        # ----------------------------------------------------------------
        # Now find changepoints by iteratively peeling off the last block
        # ----------------------------------------------------------------
        change_points = np.zeros(N, dtype=int)
        i_cp = N
        ind = N
        while True:
            i_cp -= 1
            change_points[i_cp] = ind
            if ind == 0:
                break
            ind = last[ind - 1]
        change_points = change_points[i_cp:]

        return self.mjd[change_points]

#export
class LikelihoodFitness(CountFitness):
    """ Fitness function that uses the full likelihood
    """
    
    def __init__(self, lc,  p0=0.05, npt=50):
        self.npt = npt
        super().__init__(lc, p0)
        
    def setup(self):
        df = self.df
        N = self.N
        
        def liketable(prep):
            return prep.create_table(self.npt)
        
        self.tables = df.fit.apply(liketable).values

    def __str__(self):
        return f'{self.__class__.__name__}: {self.N} cells,  prior={self.ncp_prior:.1f}'

    def __call__(self, R):
        
        a, y  = self.tables[R]
        x = np.linspace(*a)
        y = np.zeros(self.npt)
        rv = np.empty(R+1)
        for i in range(R, -1, -1): 
            a, yi = self.tables[i]
            xi = np.linspace(*a)
            y += np.interp(x, xi, yi, left=-np.inf, right=-np.inf)
            amax = np.argmax(y)
            rv[i] =y[amax]
        return rv    

In [ ]:
%nbdev_collapse_input
lcs ={} # make small light curves available locally

def data_setup(config, lcs=lcs, mjd_query='54750<t<54855', names=['Geminga','3C 279']):
    """
    ### Generate data sets for an AGN and a pulsar
    {printout}
    
    Choose the time interval, {mjd_query} ({days} days) to bracket a modest flare of the AGN.
     
    <table>
    <tr> <td>Pulsar</td><td>AGN</td></tr>
    <tr>
    <td>{fig1}</td> <td>{fig2}</td>
    </tr>
    </table>
    """
    
    from wtlike.config import Config,  PointSource
    from wtlike.lightcurve import  getLightCurve, flux_plot
    figs=[]
    plt.rc('font', size=20)
    with capture_print('printout') as printout:
        for i,name in enumerate(names):
            lcfull = getLightCurve(config,  PointSource(name), key=None)
            lc = lcs[name] = lcfull.query(mjd_query) if mjd_query else lcfull
            fig= flux_plot(config,lc, fignum=i, title=name)
            figs.append(figure(fig, width=300))
    fig1, fig2 = figs
    mjd_query = mjd_query.replace('<', '&lt;')
    days = len(lc)
    return locals()

config = Config(data_folder='/home/burnett/monthly')
if config.valid:
    nbdoc(data_setup, config, lcs, mjd_query='54750<t<54855', )
else:
    print('No analysis: config not valid')

### Generate data sets for an AGN and a pulsar
<details  class="nbdoc-description" >  <summary> printout </summary>  <div style="margin-left: 5%"><pre>photons and exposure for Geminga: Restoring from cache with key "Geminga_monthly_data"<br>Time bins: 4624 intervals of 1 days, from MJD 54683.0(2008-08-05) to 59307.0(2021-04-03))<br>Loaded 4424 / 4424 cells with exposure &gt; 0.3 for light curve analysis<br>photons and exposure for 3C 279: Restoring from cache with key "3C 279_monthly_data"<br>Time bins: 4624 intervals of 1 days, from MJD 54683.0(2008-08-05) to 59307.0(2021-04-03))<br>Loaded 4422 / 4422 cells with exposure &gt; 0.3 for light curve analysis<br></pre></div> </details>

Choose the time interval, 54750&lt;t&lt;54855 (105 days) to bracket a modest flare of the AGN.
 
<table>
<tr> <td>Pulsar</td><td>AGN</td></tr>
<tr>
<td><div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/data_setup_fig_02.png" title="images/data_setup_fig_02.png">    <img src="images/data_setup_fig_02.png" alt="Figure 2 at images/data_setup_fig_02.png" width=300>   </a> </figure>
</div>
</td> <td><div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 3">  <a href="images/data_setup_fig_03.png" title="images/data_setup_fig_03.png">    <img src="images/data_setup_fig_03.png" alt="Figure 3 at images/data_setup_fig_03.png" width=300>   </a> </figure>
</div>
</td>
</tr>
</table>


In [ ]:
%nbdev_export
def doc_countfitness( fitness, light_curve_dict, source_name):
    """
    #### {class_name} test with source {source_name}
         
    Create object: `bbfitter = {class_name}(lc)`
    
    Object description:   {bbfitter}
    
    Then `bbfitter({n})` returns the values
        {values}
   
    Finally, the partition algorithm, 'bbfitter.fit()' returns {cffit}
    
    """
    
    lc = light_curve_dict[source_name]
    bbfitter = fitness(lc)
    class_name = bbfitter.name
    n = 10
    values  = np.array(bbfitter(n)).round(1)    
    cffit = bbfitter.fit()
    
    return locals()

In [ ]:
%nbdev_collapse_input
if config.valid:
    nbdoc(doc_countfitness, CountFitness, light_curve_dict = lcs, source_name='Geminga')
    nbdoc(doc_countfitness, CountFitness, light_curve_dict = lcs, source_name='3C 279')
else:
     print('No analysis: config not valid')
     

#### CountFitness test with source Geminga
     
Create object: `bbfitter = CountFitness(lc)`

Object description:   CountFitness: 105 cells, spanning 133.4 days, prior=4.9

Then `bbfitter(10)` returns the values
    [19135.3 17237.9 15427.8 13421.1 12009.5 10042.1  8595.7  6817.3  5235.3
  3546.6  1889.7]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54792. 54855.]


#### CountFitness test with source 3C 279
     
Create object: `bbfitter = CountFitness(lc)`

Object description:   CountFitness: 105 cells, spanning 139.0 days, prior=4.9

Then `bbfitter(10)` returns the values
    [2572.9 2266.5 2004.3 1767.5 1452.8 1265.2 1018.7  846.1  667.2  466.1
  250.3]

Finally, the partition algorithm, 'bbfitter.fit()' returns [54750. 54754. 54782. 54790. 54807. 54827. 54855.]


In [ ]:
%nbdev_export
def get_bb_partition(config, lc, fitness_class=LikelihoodFitness, p0=0.05, key=None, clear=False):    

    """Perform Bayesian Block partition of the cells found in a light curve
    
    - lc : input LightCurve object or DataFrame with fit cells
    - fitness_class 
    
    return edges for partition
    """
    assert issubclass(fitness_class,CountFitness), 'fitness_class wrong'
    if not isinstance(lc, pd.DataFrame):
        lc = lc.dataframe
    assert 'fit' in lc.columns, 'Expect the dataframe to have the Poisson fit object'


    def doit():
        fitness = fitness_class(lc, p0=p0)
        # Now run the astropy Bayesian Blocks code using my version of the 'event' model
        return fitness.fit() 
        
    key = f'BB_edges_' if key is '' else key
    
    edges = config.cache(key, doit,  description=key if config.verbose>0 else '', overwrite=clear)
    
    if config.verbose>0:
        print(f'Partitioned {len(lc)} cells into {len(edges)-1} blocks, using {fitness_class.__name__} ' )
    return edges

In [ ]:
show_doc(get_bb_partition, title_level=2)

<h2 id="get_bb_partition" class="doc_header"><code>get_bb_partition</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>get_bb_partition</code>(**`config`**, **`lc`**, **`fitness_class`**=*`'LikelihoodFitness'`*, **`p0`**=*`0.05`*, **`key`**=*`None`*, **`clear`**=*`False`*)

Perform Bayesian Block partition of the cells found in a light curve

- lc : input LightCurve object or DataFrame with fit cells
- fitness_class 

return edges for partition

In [ ]:
%nbdev_collapse_input

def test_bb(lcs, name, fitness):
    """
    ### BB partition for {name} using {fitness.__name__}
    
    {lc_fig}
    """
    lc = lcs[name]
    edges = get_bb_partition(config, lc, fitness, key=None) 
    lc_fig = flux_plot(config, lc, title=f'{name} partition with {fitness.__name__}')
    lc_fig.width=400
    ax = lc_fig.axes[0]
    edges = np.concatenate([edges, [edges[-1]] ])
    for  i,t in enumerate(edges[::2]):
        if 2*i+1==len(edges): break
        t2 = edges[2*i+1]
        ax.axvspan(t, t2, color='lightcyan')
    for t in edges:
        ax.axvline(t, ls=':', color='cyan')
    return locals()

if config.valid:
    nbdoc(test_bb, lcs, '3C 279', CountFitness, name='count')
    nbdoc(test_bb, lcs, '3C 279', LikelihoodFitness, name='like')
else:
    print('No output, config not valid')
#     nbdoc(test_bb, lcs, 'Geminga', CountFitness)
#     nbdoc(test_bb, lcs, 'Geminga', LikelihoodFitness)

Partitioned 105 cells into 6 blocks, using CountFitness 


### BB partition for 3C 279 using CountFitness

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/count_fig_01.png" title="images/count_fig_01.png">    <img src="images/count_fig_01.png" alt="Figure 1 at images/count_fig_01.png" width=400>   </a> </figure>
</div>



Partitioned 105 cells into 6 blocks, using LikelihoodFitness 


### BB partition for 3C 279 using LikelihoodFitness

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/like_fig_01.png" title="images/like_fig_01.png">    <img src="images/like_fig_01.png" alt="Figure 1 at images/like_fig_01.png" width=400>   </a> </figure>
</div>



In [ ]:
%nbdev_export
def bb_overplot(config, lc, bb_fit, ax=None, **kwargs):
    """Plot light curve: cell fits with BB overplot
    """
    import matplotlib.pyplot as plt
    colors = kwargs.pop('colors', ('lightblue', 'wheat', 'blue'))
    fig, ax = plt.subplots(1,1, figsize=(12,4)) if not ax else (ax.figure, ax)
    flux_plot(config, lc, ax=ax, colors=colors,**kwargs)
    flux_plot(config, bb_fit, ax=ax, step=True, **kwargs)

In [ ]:
show_doc(bb_overplot,title_level=2)

<h2 id="bb_overplot" class="doc_header"><code>bb_overplot</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h2>

> <code>bb_overplot</code>(**`config`**, **`lc`**, **`bb_fit`**, **`ax`**=*`None`*, **\*\*`kwargs`**)

Plot light curve: cell fits with BB overplot
    

In [ ]:
%nbdev_export
class BayesianBlockAnalysis(CellData):
    """Full analysis of a source, including Bayesian block analysis
    Inherits from `CellData`
    
    - source
    - exp_min
    - bins
    - clear -- if set, clear the cache entry
    """
    def __init__(self, config, source, exp_min=0.3, bins=None,  clear=False):
        super().__init__(config, source, exp_min, bins, clear)
        # get the cells from superclass
        self.cells = self.dataframe
        self.lc = LightCurve(config, self.cells, source)
        self.lc_df = self.lc.dataframe
        
    def partition(self,key=None, clear=False):
    
        key = self.source.data_key.replace('data', 'bb_edges') if key is None else key
        self.source.edges_key = key
        bb_edges  = get_bb_partition(self.config, self.lc_df,  key=key, clear=clear) 

        self.bb_cells = partition_cells(self.config, self.cells, bb_edges);
        self.bb_fit = fit_cells(self.config, self.bb_cells, )
        
    def plot(self, ax=None, **kwargs):
        import matplotlib.pyplot as plt
        fig, ax = ig, ax = plt.subplots(figsize=(12,3)) if ax is None else (ax.figure, ax)
        bb_overplot(self.config, self.lc_df, self.bb_fit, ax=ax, **kwargs)
        return fig

    
# alias        
BBA = BayesianBlockAnalysis

In [ ]:
show_doc(BayesianBlockAnalysis)

<h2 id="BayesianBlockAnalysis" class="doc_header"><code>class</code> <code>BayesianBlockAnalysis</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>BayesianBlockAnalysis</code>(**`config`**, **`source`**, **`exp_min`**=*`0.3`*, **`bins`**=*`None`*, **`clear`**=*`False`*) :: [`CellData`](wtlike/cell_data#CellData)

Full analysis of a source, including Bayesian block analysis
Inherits from [`CellData`](wtlike/cell_data#CellData)

- source
- exp_min
- bins
- clear -- if set, clear the cache entry

### Fit blocks and overplot

The following code shows how to refit, and demonstrates the overplot function.

In [ ]:
lcs['3C 279']

,t,tw,n,e,fit
67,54750.5,1.0,76,1.29,"0.526[1+0.318-0.375], < 0.90"
68,54751.5,1.0,67,1.32,"0.496[1+0.311-0.369], < 0.85"
69,54752.5,1.0,62,1.35,"0.302[1+0.445-0.556], < 0.64"
70,54753.5,1.0,79,1.40,"0.712[1+0.259-0.296], < 1.10"
71,54754.5,1.0,52,1.41,"0.428[1+0.347-0.410], < 0.76"
...,...,...,...,...,...
167,54850.5,1.0,50,1.10,"0.908[1+0.237-0.260], < 1.33"
168,54851.5,1.0,48,1.00,"0.940[1+0.253-0.290], < 1.45"
169,54852.5,1.0,55,1.11,"0.955[1+0.238-0.260], < 1.40"
170,54853.5,1.0,54,1.01,"0.958[1+0.249-0.286], < 1.47"


In [ ]:
%nbdev_collapse_input
lc = lcs['3C 279']
# todo -- test with this
# if config.valid:
#     name = '3C 279'
#     config.verbose=1
#     source = PointSource(name)
#     bba = BBA(config, source)
#     bba.partition()
#     fig = bba.plot(yscale='log')
#     plt.gca().set_title(f'Bayesian-block analysis of {name}')
# else:
#     print('Not showing BB plots')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_gti.ipynb.
Converted 02_source_data.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cell_data.ipynb.
Converted 07_cells.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Sun Apr 25 16:21:56 PDT 2021
